In [7]:
import csv
from calculos_tarifas import *

# Función para cargar tarifas desde un archivo CSV
def cargar_tarifas():
    tarifas = []
    with open('tarifas_nosolar.csv', 'r') as archivo_csv:
        lector = csv.DictReader(archivo_csv)
        for fila in lector:
            tarifa = {
                "compañia": fila['Empresa'],
                "nombre": fila['Tarifa'],
                "punta": float(fila['Punta']),
                "llano": float(fila['Llano']),
                "valle": float(fila['Valle']),
                "P1": float(fila['P1']) if 'P1' in fila else 0,
                "P3": float(fila['P3']) if 'P3' in fila else 0,
                #"excedente": float(fila['Excedentes']) if fila['Excedentes'] else 0,
                "batería": 0
                #"otros": float(fila['Coste de gestión'])
            }
            tarifas.append(tarifa)
    return tarifas



def obtener_incremento_por_habitante(num_personas):
    if num_personas == 1:
        return 1.0
    elif num_personas == 2:
        return 1.25
    elif num_personas == 3:
        return 1.45
    elif num_personas == 4:
        return 1.60
    else:
        return 1.60 + 0.1 * (num_personas - 4)



# Función para ajustar el consumo en función del número de habitantes
def ajustar_consumo_por_habitantes(consumo, num_personas):
    # Aplica el factor de incremento por cada persona adicional
    incremento = obtener_incremento_por_habitante(num_personas)
    return consumo * incremento
 
 

# Formulario para que el cliente ingrese datos de consumo
def formulario_consumo():
    print("¡Bienvenido a la comparativa de tarifas eléctricas!")
    mes = input("¿Para qué mes deseas hacer la comparativa?: ").lower()
    provincia = input("¿En qué provincia te encuentras?: ").lower()
    num_personas = int(input("¿Cuántas personas viven en tu domicilio?: "))
    
    conoce_consumo = input("¿Conoces tu consumo en kWh para los periodos de Valle, Llano y Punta? (s/n): ").strip().lower()
    
    if conoce_consumo == 's':
        try:
            consumo_valle = float(input("Introduce tu consumo en kWh para el periodo Valle: "))
            consumo_llano = float(input("Introduce tu consumo en kWh para el periodo Llano: "))
            consumo_punta = float(input("Introduce tu consumo en kWh para el periodo Punta: "))
            potencia = float(input("Introduce tu potencia en kW: "))

              # Ajustar el consumo en función del número de habitantes
            consumo_valle = ajustar_consumo_por_habitantes(consumo_valle, num_personas)
            consumo_llano = ajustar_consumo_por_habitantes(consumo_llano, num_personas)
            consumo_punta = ajustar_consumo_por_habitantes(consumo_punta, num_personas)
            
            return {"valle": consumo_valle, "llano": consumo_llano, "punta": consumo_punta, "potencia": potencia, "mes": mes}
        except ValueError:
            print("Por favor, introduce valores numéricos.")
            return formulario_consumo()
    
    elif conoce_consumo == 'n':
        consumo_estimado = 0
        electrodomesticos = {
            "Frigorífico": 1.2, "Lavadora": 0.9, "Lavavajillas": 0.85,
            "Televisor": 0.2, "Aire Acondicionado": 3.0, "Vitroceramica": 1.5,
            "Horno": 1.0, "Calefactores eléctricos": 2.0, "Microondas": 0.2,
            "Ordenador": 0.5, "Plancha de ropa": 0.3, "Otros": 0.5
        }
        
        print("Vamos a estimar tu consumo. Por favor, indica si tienes los siguientes electrodomésticos:")
        for electrodomestico, consumo in electrodomesticos.items():
            respuesta = input(f"¿Tienes {electrodomestico}? (s/n): ").strip().lower()
            if respuesta == 's':
                consumo_estimado += consumo

        consumo_valle = consumo_estimado * 0.4 * 30
        consumo_llano = consumo_estimado * 0.3 * 30
        consumo_punta = consumo_estimado * 0.3 * 30
        potencia = float(input("Introduce tu potencia en kW: "))

    # Ajuste del consumo basado en habitantes
        consumo_valle = ajustar_consumo_por_habitantes(consumo_valle, num_personas)
        consumo_llano = ajustar_consumo_por_habitantes(consumo_llano, num_personas)
        consumo_punta = ajustar_consumo_por_habitantes(consumo_punta, num_personas)


        print("\nEstimación de consumo:")
        print(f"Consumo estimado en Valle: {consumo_valle:.2f} kWh")
        print(f"Consumo estimado en Llano: {consumo_llano:.2f} kWh")
        print(f"Consumo estimado en Punta: {consumo_punta:.2f} kWh")

        return {"valle": consumo_valle, "llano": consumo_llano, "punta": consumo_punta, "potencia": potencia, "mes": mes}

    else:
        print("Opción no válida. Por favor, introduce 's' o 'n'.")
        return formulario_consumo()






# Diccionario de meses y días
dias_por_mes = {
    "enero": 31, "febrero": 28, "marzo": 31, "abril": 30,
    "mayo": 31, "junio": 30, "julio": 31, "agosto": 31,
    "septiembre": 30, "octubre": 31, "noviembre": 30, "diciembre": 31
}

# Comparativa de tarifas
def calcular_mejor_tarifa(datos_consumo, tarifas):
    mes = datos_consumo["mes"]
    if mes is None:
        print("El mes no ha sido proporcionado correctamente.")
        return
    dias = dias_por_mes.get(mes.lower(), 30)  # Usar 30 días si el mes es inválido o no está en el diccionario
    excedentes = 0
    potencia = datos_consumo['potencia']
    iva = 1.21
    bono_social = 0.006282
    impuesto = 3.8 / 100
    equipos = 0.82

    comparativa = []
    for empresa in tarifas:
        consumo_total = (empresa["punta"] * datos_consumo["punta"] +
                         empresa["llano"] * datos_consumo["llano"] +
                         empresa["valle"] * datos_consumo["valle"])
        potencia_total = potencia * dias * (empresa["P1"] + empresa["P3"])
        #excedentes_total = excedentes * empresa["excedente"]
        
        #precio = ((potencia_total + bono_social * dias + consumo_total) * (1 + impuesto) +
                  #equipos + empresa["batería"] * dias) * iva - excedentes_total
        
        #precio_sin = ((potencia_total + bono_social * dias + max(consumo_total - excedentes_total, 0)) * (1 + impuesto) +
                      #equipos + empresa["batería"] * dias) * iva

        precionormal = ((potencia_total + bono_social * dias + consumo_total) * (1 + impuesto) +
                        equipos) * iva 
        
        calculado = {
            "Empresa": empresa["compañia"],
            "Tarifa": empresa["nombre"],
            "Precio": precionormal
        }
        comparativa.append(calculado)

    comparativa.sort(key=lambda x: x["Precio"])
    mejor_tarifa = comparativa[0]

    print("\nComparativa de tarifas:")
    for tarifa in comparativa:
        print(f"{tarifa['Empresa']} - {tarifa['Tarifa']}: {tarifa['Precio']:.2f} €")

    print(f"\nLa tarifa más económica es la de {mejor_tarifa['Empresa']} - {mejor_tarifa['Tarifa']} con un precio de {mejor_tarifa['Precio']:.2f} €.")

# Ejecución del formulario y cálculo de la mejor tarifa
tarifas = cargar_tarifas()
datos_consumo = formulario_consumo()
calcular_mejor_tarifa(datos_consumo, tarifas)

¡Bienvenido a la comparativa de tarifas eléctricas!
Vamos a estimar tu consumo. Por favor, indica si tienes los siguientes electrodomésticos:

Estimación de consumo:
Consumo estimado en Valle: 233.28 kWh
Consumo estimado en Llano: 174.96 kWh
Consumo estimado en Punta: 174.96 kWh

Comparativa de tarifas:
Octopus - Octopus Relax: 100.63 €
Octopus - Octopus 3: 102.46 €
Naturgy - Por uso de Luz: 103.55 €
Xenera - Home contigo: 104.29 €
Octopus - Octopus Flexi: 105.16 €
Xenera - Home contigo única: 105.97 €
Naturgy - Noche Luz: 106.34 €
Repsol - Tarifa Ahorro Plus: 106.93 €
Xenera - A vaquiña fija: 111.39 €
Repsol - Tarifa Tranquilisima: 114.37 €
Xenera - En casa: 119.53 €
Repsol - Tarifa 10h descuento: 120.50 €

La tarifa más económica es la de Octopus - Octopus Relax con un precio de 100.63 €.
